In [60]:
import requests, datetime, pickle

URL = 'https://open-api.bingx.com'

descargar_data = False

# Obtener la serie de precios para BTC, ETH y XRP 

En temporalidades: 1d y/o 4h, 2h, 1h, 30m, 15m

data_raw = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
}<br />

In [19]:
from backtesting.fechas import past_timestamp, now_timestamp, retrasar_tres_horas
from backtesting.data import response_to_dataframe
from backtesting.ficheros import crear_directorio

In [9]:
if not descargar_data:
    # open pickle file storaged in ./data/data_raw.pickle
    with open('./data/data_raw.pickle', 'rb') as f:
        data_raw = pickle.load(f)

else:
    path = '/openApi/swap/v2/quote/klines'

    data_1d = True
    data_low_temporality = True

    temporalidades = []
    if data_1d:
        temporalidades.append('1d')
    if data_low_temporality:
        temporalidades.append('4h')
        temporalidades.append('2h')
        temporalidades.append('1h')
        temporalidades.append('30m')
        temporalidades.append('15m')
    if not data_1d and not data_low_temporality:
        exit()

    now = datetime.datetime.now()
    start_time = []
    for t in temporalidades:
        if t == '1d':
            start_time.append(int(past_timestamp(365, 'days', now)))
        elif t == '4h':
            start_time.append(int(past_timestamp(45, 'days', now)))
        elif t == '2h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '1h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '30m':
            start_time.append(int(past_timestamp(6, 'days', now)))
        elif t == '15m':
            start_time.append(int(past_timestamp(3, 'days', now)))

    limit = 555
    activos = ['BTC', 'ETH', 'XRP']
    service = URL + path
    endTime = int(datetime.datetime.timestamp(datetime.datetime.now()) * 1000)
    data_raw = {}
    for activo in activos:
        data_raw[activo] = {}
        symbol = activo + '-USDT'
        for i, interval in enumerate(temporalidades):
            response = requests.get(service, params={'symbol':symbol, 'interval':interval, 'limit':limit, 'startTime':start_time[i], 'endTime':endTime})
            df = response_to_dataframe(response.json())
            if df is None:
                continue
            df['time'] = df['time'].apply(retrasar_tres_horas)
            df = df.set_index('time')
            data_raw[activo][interval] = df

    crear_directorio('data')
    with open('./data/data_raw.pickle', 'wb') as f:
        pickle.dump(data_raw, f, pickle.HIGHEST_PROTOCOL)

# Aplicar Squeeze Momentum Indicator a cada serie temporal y obtener la temporalidad que tenga mayor correlacion para cada activo

SMI_default = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
<br />
}<br />
<br />
SMI_default_corr = {<br />
    'BTC': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'ETH': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'XRP':<br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    }<br />
<br />
}<br />

In [36]:
from backtesting.indicadores import squeeze_momentum_indicator
from scipy.stats import pearsonr

In [30]:
SMI_default = {}
for activo in activos:
    SMI_default[activo] = {}
    for t in temporalidades:
        df = data_raw[activo][t].iloc[:]
        df = squeeze_momentum_indicator(df)
        SMI_default[activo][t] = df.dropna()
        

In [42]:
SMI_default_corr = {}
for activo in activos:
    SMI_default_corr[activo] = {}
    for t in temporalidades:
        df = SMI_default[activo][t].iloc[:]
        df = df[['value', 'close']]
        SMI_default_corr[activo][t], _ = pearsonr(df['value'], df['close'])

print('Correlaciones')
for activo in activos:
    for t in temporalidades:
        print(f'{activo} {t} {round(SMI_default_corr[activo][t], 2)}')
    print()

Correlaciones
BTC 1d 0.36
BTC 4h 0.57
BTC 2h 0.45
BTC 1h 0.35
BTC 30m 0.07
BTC 15m 0.71

ETH 1d 0.41
ETH 4h 0.47
ETH 2h 0.5
ETH 1h 0.5
ETH 30m 0.21
ETH 15m 0.65

XRP 1d 0.41
XRP 4h 0.34
XRP 2h 0.39
XRP 1h 0.27
XRP 30m 0.05
XRP 15m 0.63



In [52]:
temporalidades_filtradas = {}
for activo in activos:
    temporalidades_filtradas[activo] = {}
    for t in temporalidades:
        if SMI_default_corr[activo][t] > 0.5:
            temporalidades_filtradas[activo][t] = SMI_default_corr[activo][t]

# print('Temporalidades filtradas')
# for activo in activos:
#     for t in temporalidades:
#         print(f'{activo} {round(temporalidades_filtradas[activo][t], 2)}')
#     print()

print('Temporalidades filtradas')
for activo in temporalidades_filtradas:
    print(activo)
    for t in temporalidades_filtradas[activo]:
        print(f'correlacion {t}: {round(temporalidades_filtradas[activo][t], 2)}')
    print()

Temporalidades filtradas
BTC
correlacion 4h: 0.57
correlacion 15m: 0.71

ETH
correlacion 2h: 0.5
correlacion 1h: 0.5
correlacion 15m: 0.65

XRP
correlacion 15m: 0.63



### conclusiones 1:

- Si optimizo los parámetros para 1d, como afectan las demás temporalidades?
- Si optimizo los parámetros para una temporalidad con alta correlación, como afectan las demás temporalidades?
- Que diferencia existe entre aplicar la estrategia con parámetros por default y optimizados?

# Aplicar las condiciones de la estrategia a las temporalidades más correlacionadas y visualizar las últimas entradas

In [53]:
from backtesting.indicadores import backtest_entry

In [55]:
entradas_backtest = {}
for activos in temporalidades_filtradas:
    for t in temporalidades_filtradas[activos]:
        entrada_short, entrada_long = backtest_entry(SMI_default[activos][t])
        dict_key = activos + '_' + t
        if len(entrada_short) > 0:
            dict_key_2 = dict_key + '_short'
            entradas_backtest.update({dict_key_2: entrada_short})
        if len(entrada_long) > 0:
            dict_key_2 = dict_key + '_long'
            entradas_backtest.update({dict_key_2: entrada_long})
        

In [59]:
# tomamos las 10 últimas 
entradas_backtest.keys()
entradas_backtest['BTC_15m_short']

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-05-09 03:15:00,27471.0,27474.5,27509.5,27451.5,355.06,27695.175534,27486.744466,58.0,27691.035395,27490.884605,-97.752536,False
2023-05-09 20:00:00,27568.6,27588.3,27604.3,27567.6,152.66,27688.471092,27508.408908,36.7,27687.946344,27508.933656,-41.196250,False
2023-05-09 21:15:00,27645.6,27760.5,27774.7,27637.0,639.30,27714.426085,27513.903915,137.7,27702.554440,27525.775560,-17.639964,False
2023-05-10 03:30:00,27596.1,27606.2,27622.7,27590.0,208.28,27732.755801,27599.694199,32.7,27726.886333,27605.563667,-77.680250,False
2023-05-11 00:15:00,27415.8,27438.5,27442.0,27365.0,802.46,27669.480124,27424.279876,77.0,27666.957066,27426.802934,-85.820607,False
2023-05-11 06:00:00,27385.3,27383.6,27398.1,27345.8,332.89,27557.154350,27362.475650,52.3,27555.062425,27364.567575,-7.948179,False
2023-05-11 11:45:00,27116.4,27114.1,27148.8,26949.2,2288.48,27604.591085,27128.518915,199.6,27592.148637,27140.961363,-148.658107,False


# Optimizar los parámetros del SMI para cada activo en la temporalidad de mayor correlación

SMI_optimized = {<br />
    'BTC': <br />
    {'best_temporality': pd.DataFrame()},<br />
    'ETH': <br />
    {'best_temporality': pd.DataFrame()},<br />
    'XRP':<br />
    {'best_temporality': pd.DataFrame()}<br />
<br />
}<br />

# Aplicar los criterios de la estrategia para cada configuración del indicador y visualizar las señales